# SPIRA INFERENCE SYSTEM MODEL TEMPLATE

## Dependencies

In [3]:
import mlflow
from mlflow.pyfunc import PythonModel
# only necessary if environment variables were not set yet.
import os

## Model Class Template

In [7]:
from typing import List, Tuple


class ModelTemplate(PythonModel):

    def load_context(self, context) -> None:
      """
        This method is called as soon as the model is instantiated.

        The same context will also be available during calls to predict, 
        but it may be more efficient to override this method and load 
        artifacts from the context at model load time.

        :param context: A :class:`~PythonModelContext` instance containing artifacts that the model
                        can use to perform inference.
      """
      pass

    def predict(self, context, model_input) -> Tuple[List[float],str]:
      """
      Evaluates the inference files and metadata and returns a prediction.

      :param context: A :class:`~PythonModelContext` instance containing artifacts that the model
                      can use to perform inference.
      :param model_input: A list of two dictionaries, the first containing inference metadata and
                      the second containing audio file bytearrays.
      
      returns: A Tuple where the first element is a list of numbers and the second is the diagnosis
      string ("positive", "negative", "inconclusive"). 
      """
      pass

## I/O Examples

## Setting the credentials

In [5]:
def setup_credentials():
  AWS_ACCESS_KEY_ID = "access-key-in-prod"
  AWS_SECRET_ACCESS_KEY = "secret-acess-key-in-prod"
  MLFLOW_S3_ENDPOINT_URL = "artifact-storage-minio-endpoint"

  os.environ["AWS_ACCESS_KEY_ID"] = AWS_ACCESS_KEY_ID
  os.environ["AWS_SECRET_ACCESS_KEY"] = AWS_SECRET_ACCESS_KEY
  os.environ["MLFLOW_S3_ENDPOINT_URL"] = MLFLOW_S3_ENDPOINT_URL

setup_credentials()

## Registering the model

In [9]:
def register_model():
  MLFLOW_CONN_URL = "connection-url-of-mlflow-server"

  # connect to mlflow server
  mlflow.set_tracking_uri(MLFLOW_CONN_URL)

  # instantiates model
  model = ModelTemplate()

  # register model
  mlflow.pyfunc.log_model(
    artifact_path="path-in-minio-to-model-artifacts",
    registered_model_name="model-name",
    python_model=model
  )

register_model()


The model should appear in the MLFlow server, where you will able to see the past versions and also control the staging and production versions of it.

## Registering the model in the API Database

The model now needs to be registered in the system database so that it becomes accessible from the front-end.

Send a POST request (it can be thorugh a script or an app like Postman) to the endpoint `/models` of the inference system API, specifying the model `name` and `publishing_channel` in the request body.
The name is the name to be displayed in the front-end, while publishing channel is the NATS topic in which the API will send messages so that the Model Server can listen to it and make the predictions. 

Note: you need user credentials to make the request.

## Deploying the Model

Once the model is registered in the database, the only step left is to lift a server with the model. In order to do that, create a new directory in the machine where the API is hosted and clone the GitHub repository SPIRA-Inference-System and fill the environment variables.

Particularly, in the file `server_envs/message_listener.env`, the `receiving_channel` should be the `publishing channel` that you registered in the database while `central_channel` should be the channel where the results are sent to the API (this channel is a convention of the deployed system). Be sure that the channels are correct, otherwise the model will not receive the requests.

The `model_path` variable specified in `server_envs/model_register.env` is the path of the model you registered in MLFlow. The path follows the convention `models:/<model-name>/<model-version>`. Be sure that you are deploying the right model and the right version of it.

Specify the name of the service at `docker-compose.server.yml` as the name of the model you registered in the database and run `bash build-server.sh`.

## Well Done!
Your model is now available for use :)